#Adversarial examples

### Libraries

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Chargement des données MNIST

In [ ]:
# load MNIST dataset
def load_mnist(split, batch_size):
  train = True if split == 'train' else False
  dataset = datasets.MNIST("./data", train=split, download=True, transform=transforms.ToTensor())
  return DataLoader(dataset, batch_size=batch_size, shuffle=train)

batch_size = 100
train_loader = load_mnist('train', batch_size)
test_loader = load_mnist('test', batch_size)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


### Implementer la classe ProjectedGradientDescent.

Vous devez implementer la fonction "compute" qui génère et retourne à partir de $(x,y)$ un exemple adversarial $\tilde{x} = x + \delta$ tel que model$(\tilde{x})\neq y$ en utilisant la methode PGD.

This is gradient ascent since we are maximizing a function rather than minimization, it is just the obvious choice for how we might maximize the inner objective a bit more carefully than with FGSM.

In [ ]:
def pgd(model, X, y, epsilon=0.8, alpha=0.08, num_iter=20):
  
    delta = torch.zeros_like(X, requires_grad=True) # commence à un point 0
        
    for t in range(num_iter):
        loss = nn.CrossEntropyLoss()(model(X + delta), y) # fonction de coût à utiliser
        loss.backward() # calculer le gradient avec les dérivés partiels
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon) # Afin de maximiser les coûts, on va à ajuster le delta dans le sens du gradient
        delta.grad.zero_()

    return delta.detach() #en détachant les gradients précédents

### Construire l'architecture de votre modèle

Vous devez construire un reseau de neurones convolutionnel.

In [ ]:
torch.manual_seed(0)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)    

model_cnn = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), 
                          nn.ReLU(),
                          nn.Conv2d(32, 32, 3, padding=1, stride=2), 
                          nn.ReLU(),
                          nn.Conv2d(32, 64, 3, padding=1), 
                          nn.ReLU(),
                          nn.Conv2d(64, 64, 3, padding=1, stride=2), 
                          nn.ReLU(),
                          Flatten(),
                          nn.Linear(7*7*64, 100), 
                          nn.ReLU(),
                          nn.Linear(100, 10)).to(device)

### Créer la fonction d'entrainement.

On remarque que la fonction "training" prend en argument "attack". Si "attack" est égal à None, l'entrainement se fait sur les données naturelles (non-attaquées). A l'inverse, si "attack" est une instance de classe d'attaque (ex. ProjectedGradientDescent) alors vous devez entrainer votre modèle sur des examples attaqué grace à l'instance "attack".

In [ ]:
def training(loader, model, opt=None, attack= None):
    total_loss, total_err = 0.,0. # definir les variables de Loss et Error
    for X,y in loader: # boucle sur input et output 
        X,y = X.to(device), y.to(device)
        if attack == None: # on va faire le training du model sans le delta (bruit)
          yp = model(X)
          criterion = nn.CrossEntropyLoss()(yp,y) # fixer la fonction de coût comme critère
        else:
          delta = attack(model, X, y) 
          yp = model(X+delta) # on va faire le training du model avec le delta (bruit)
          criterion = nn.CrossEntropyLoss()(yp,y)
        if opt: # si nous avons un optimizateur à utiliser
          opt.zero_grad()
          criterion.backward()
          opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item() # mettre à jour l'erreur
        total_loss += criterion.item() * X.shape[0] # mettre à jour le coût
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [ ]:
opt = optim.SGD(model_cnn.parameters(), lr=1e-1)
for _ in range(5):
    train_err, train_loss = training(train_loader, model_cnn, opt)
    test_err, test_loss = training(test_loader, model_cnn)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")

0.017000	0.054547	0.011550	0.036093
0.012567	0.040191	0.008600	0.028698
0.009267	0.030399	0.006750	0.022175
0.007567	0.023617	0.006933	0.021166
0.006167	0.020347	0.004733	0.014327


### Implémenter la fonction d'évalutation. 

Cette fonction doit renvoyer l'accuracy du modèle. Si attack est égal à None, vous devez renvoyer l'accuracy du modèle testé sur des données naturelle (non-attaquées). Si attack est different de None, vous devez renvoyer l'accuracy du modèle testé sur des données attaquée.

In [ ]:
def eval_model(model, loader, attack = None):
    total_loss, total_err = 0.,0. # definir les variables de Loss et Error
    for X,y in loader: # boucle sur input et output 
        X,y = X.to(device), y.to(device)
        if attack == None: # on va faire le training du model sans le delta (bruit)
          yp = model(X)
          criterion = nn.CrossEntropyLoss()(yp,y) # fixer la fonction de coût comme critère
        else:
          delta = attack(model, X, y) 
          yp = model(X+delta) # on va faire le training du model avec le delta (bruit)
          criterion = nn.CrossEntropyLoss()(yp,y)
        
        total_err += (yp.max(dim=1)[1] != y).sum().item() # mettre à jour l'erreur
        total_loss += criterion.item() * X.shape[0] # mettre à jour le coût
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [ ]:
evaluation_cnn = eval_model(model_cnn, test_loader)
evaluation_cnn_attack = eval_model(model_cnn, test_loader, pgd)
print("Performance CNN:", evaluation_cnn)
print("Performance attacked CNN:", evaluation_cnn_attack)

Performance CNN: (0.004733333333333333, 0.014327105870070227)
Performance attacked CNN: (0.9998833333333333, 82.21693323771159)


On peut voir ici comment le pgd a réussi à maximiser la fonction de coût atteignant un valeur de 82.22 vs 0.014 du model sans attaque et comment l'erreur a également augmenté dans le modèle attaqué le quel a le valeur de 0.99